In [3]:
%%writefile catch.py

# Import

import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException


def WC_Catch(corp=str) :
    
    # headless mode

    option = webdriver.ChromeOptions()
    option.add_argument('--headless')        # Head-less 설정
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')

    service = Service(executable_path='chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=option)

    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')

    url = r"https://www.catch.co.kr/"

    browser.maximize_window()
    print(browser.get_window_size())

    browser.get(url)



    # 로그인

    from accounts import ct

    ID = ct.ID
    PW = ct.PW



    browser.find_element(By.CSS_SELECTOR, "a.login").click()

    browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)

    browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

    browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

    time.sleep(2)



    query = browser.find_element(By.CSS_SELECTOR, "input")

    query.send_keys(corp)

    time.sleep(1)

    query.send_keys(Keys.ENTER)

    time.sleep(2)



    ### 팝업 뜨면 끄기 (예외 처리)

    try : 
        browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
        
    except : 
        pass

    time.sleep(2)



    # 검색한 기업(최상위 노출) 클릭 
    browser.find_element(By.CSS_SELECTOR, "p.name > a").click()

    time.sleep(1)

    # '현직자 리뷰' 탭 클릭
    browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()

    time.sleep(1)

    # 핵심 리뷰만 조회 - 정규직/현직자가 3년 이내 작성한 리뷰
    browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()

    time.sleep(1)





    #########################################################################

    time.sleep(1)

    # Review Crawler

    n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')

    time.sleep(1)

    good_lst = None
    bad_lst = None


    good_lst = []
    bad_lst = []


    pre_page = 0


    while True : 

        time.sleep(1)

        # 현재 페이지의 숫자
        crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
        crt_page = int(crt_btn.text)

        # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
        if crt_page == pre_page:
            break

        try:
            for p in range(1, 5) : 

                grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
                brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

                grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
                brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

                good_lst.append(grv_clean)
                bad_lst.append(brv_clean)
        
        except NoSuchElementException : 
            break

        time.sleep(2)

        # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


    # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신
            pre_page = crt_page

            # 다음 버튼 클릭
            n_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(1)

        except:
            break



    browser.close()


    # list to dataframe

    good_lst = list(set(good_lst))
    bad_lst = list(set(bad_lst))

    good_df = pd.DataFrame({"긍정적 평가" : good_lst})
    bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

    pros_cons = good_df.join(bad_df)
    pros_cons



    return pros_cons


Overwriting catch.py


In [1]:
from catch import WC_Catch

In [2]:
WC_Catch('삼성전자')

{'width': 800, 'height': 600}


,긍정적 평가,부정적 평가
0,주변 인프라 좋고 근무시간 자유롭고 예전처럼 눈치보지 않는 분위기가 정착되어가는 느낌,일이 너무 많다. 항상 일이 많다. 가끔은 일이 나를 절벽으로 몰아가고 있다는 생각...
1,"밥이 괜찮고 3끼 자 무료, 셔틀 버스 이용해서 편하게 출퇴근가능",자유롭지 못한 조직문화. 보고를 위한 보고. 사내정치 매우 강함
2,복리후생과 교육제도가 잘 되있고..여가문화도 제공한다.,개개인이 빛날 기회가 별로 없음. 능력에 비례해서 연봉이 오르진 않음.
3,극강의 안정성이 최고입니다. HR 보장이 확고하여 휴가등이 보장됩니다,부서 잘못 만나면 정시출근 야근이다. 연차 15개에 외에 창립기념일에도 안 쉼. 2...
4,"근무지역이 좋다, 성과급이나 회사를 다니며 자부심을 느낄때가있다",갈라치기 경영진 무능력 보신주의 야근 강요 문화 최악임
...,...,...
126,"자기개발을 위한 온라인 강의를 제한없이 무료로 수강할 수 있고, 출퇴근이 자유롭고,...",워라밸이 부족하며 평균 나이가 높아 적응하는데 어려움이있다
127,"근무시간 자율 출퇴근 방식으로 자유롭고, 최근 재택 근무가도 병행 가능하다. 사업부...",근속연수가 생각보다 짧음. 자기개발 안하면 뒤쳐지는 경향이 있어서 지속적으로 개발해야 함
128,여름휴가가 따로 보장되고 연차 가능하다 돈많이 벌고 싶으면 웰컴,사업부에 따른 성과급 격차가 커서 내부에서도 위화감이 있습니다
129,복지가 좋아요. 커리어 쌓기에 좋아요. 연차도 많이 써요,삼시세끼 밥을 주지만 세끼 먹을 일이 많다. 셔틀 버스가 많아서 밤에 퇴근도 많이한다.
